In [1]:
import pandas as pd

In [73]:
df = pd.read_csv("https://raw.githubusercontent.com/tantowjy/news-classification/main/dataset/talas/dataset-training-talas-830.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    833 non-null    object 
 1   link                     833 non-null    object 
 2   date                     492 non-null    object 
 3   content                  833 non-null    object 
 4   liberalism_conservative  833 non-null    int64  
 5   coalition_opposition     492 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 39.2+ KB


,title,link,date,content,liberalism_conservative,coalition_opposition
0,Wapres Harap Bendungan Cipanas Berkembang Jadi...,https://nasional.kompas.com/read/2024/07/09/20...,2024-07-09 00:00:00,Wakil Presiden (Wapres) RI Ma'ruf Amin berhara...,0,1.0
1,Mahasiswa Dikhawatirkan Terlilit Utang Jika Ba...,https://nasional.kompas.com/read/2024/07/09/20...,2024-07-09 00:00:00,Gagasan Menteri Koordinator Pembangunan Manusi...,1,0.0
2,Airlangga Nilai KPU Masih Layak Selenggarakan ...,https://nasional.kompas.com/read/2024/07/09/20...,2024-07-09 00:00:00,Ketua Umum Partai Golkar Airlangga Hartarto me...,0,1.0
3,Pakar Ragu Rencana Datangkan Dokter Asing Bisa...,https://nasional.kompas.com/read/2024/07/09/19...,2024-07-09 00:00:00,Rencana pemerintah mendatangkan dokter asing u...,1,0.0
4,Usul Muhadjir Pakai Pinjol Buat Bayar Uang Kul...,https://nasional.kompas.com/read/2024/07/09/19...,2024-07-09 00:00:00,Usul Menteri Koordinator Pembangunan Manusia d...,1,0.0
...,...,...,...,...,...,...
828,Maraknya Video Mesum Akibat Ketidaktegasan Pem...,https://www.antaranews.com/berita/208502/hti-m...,NaN,"hizbut tahrir indonesia (hti) menilai, marakny...",1,NaN
829,Harus Jadi Basis Penyelenggaraan Pemerintahan,https://www.antaranews.com/berita/206065/panca...,NaN,sehubungan peringatan hari lahir pancasila 1 j...,1,NaN
830,: Indonesia Harus Perkuat Ideologi Kebangsaan,https://www.antaranews.com/berita/181310/fpn-i...,NaN,bangsa indonesia harus mampu memperkuat ideolo...,1,NaN
831,Kiemas Ingin Parpol Nasionalis Bergabung,https://www.antaranews.com/berita/178711/taufi...,NaN,ketua presidium pusat persatuan alumni (ppa) g...,1,NaN


In [77]:
import pickle
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import pandas as pd  # Assuming you are using pandas for your dataframe

def predict_liberalism_conservative(news_texts, tokenizer_path='model/liberalism_conservative/liberalism_conservative_tokenizer.pkl', model_path='model/liberalism_conservative/liberalism_conservative.tflite', max_len=100):
    # Load tokenizer from pickle file
    with open(tokenizer_path, 'rb') as f:
        tokenizer = pickle.load(f)

    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor information
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Tokenization and padding of news texts
    sequences = tokenizer.texts_to_sequences(news_texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)

    # Check input shape compatibility
    if padded_sequences.shape[1] != input_details[0]['shape'][1]:
        raise ValueError(f"Input shape mismatch: Expected {input_details[0]['shape'][1]}, but got {padded_sequences.shape[1]}.")

    # Convert input data to float32 type
    padded_sequences = padded_sequences.astype('float32')

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], padded_sequences)

    # Run the interpreter to make predictions
    interpreter.invoke()

    # Get the prediction result from the output tensor
    predictions = interpreter.get_tensor(output_details[0]['index'])

    # Interpreting prediction results
    predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

    # Prepare the results
    results = []
    for text, pred, label in zip(news_texts, predictions, predicted_labels):
        result = {
            'Text': text,
            'Prediction': pred[0],
            'Predicted Label': "Liberal" if label == 1 else "Konservatif"
        }
        results.append(result)

    return results

# Get the list of news texts
news_texts = df['content'].tolist()

# Predict in batches (if necessary)
batch_size = 1  # Adjust the batch size as needed
results = []
for i in range(0, len(news_texts), batch_size):
    batch = news_texts[i:i+batch_size]
    results.extend(predict_liberalism_conservative(batch))

conservative_count = 0
liberal_count = 0
# Display results
for result in results:
    print(f"Prediction: {result['Prediction']:.4f}")
    print(f"Predicted Label: {result['Predicted Label']}")
    if result['Predicted Label'] == "Konservatif":
        conservative_count += 1
    else:
        liberal_count += 1
    print()

print(f"Konservatif: {conservative_count}")
print(f"Liberal: {liberal_count}")


Prediction: 0.6893
Predicted Label: Liberal

Prediction: 0.8778
Predicted Label: Liberal

Prediction: 0.7983
Predicted Label: Liberal

Prediction: 0.8512
Predicted Label: Liberal

Prediction: 0.8798
Predicted Label: Liberal

Prediction: 0.7393
Predicted Label: Liberal

Prediction: 0.6130
Predicted Label: Liberal

Prediction: 0.5506
Predicted Label: Liberal

Prediction: 0.8942
Predicted Label: Liberal

Prediction: 0.9185
Predicted Label: Liberal

Prediction: 0.8221
Predicted Label: Liberal

Prediction: 0.9149
Predicted Label: Liberal

Prediction: 0.7151
Predicted Label: Liberal

Prediction: 0.8744
Predicted Label: Liberal

Prediction: 0.9058
Predicted Label: Liberal

Prediction: 0.8822
Predicted Label: Liberal

Prediction: 0.2141
Predicted Label: Konservatif

Prediction: 0.6854
Predicted Label: Liberal

Prediction: 0.8148
Predicted Label: Liberal

Prediction: 0.6124
Predicted Label: Liberal

Prediction: 0.8875
Predicted Label: Liberal

Prediction: 0.7347
Predicted Label: Liberal

Predic